In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time
from bs4 import BeautifulSoup

### 1. Iniciamos sesión y guardamos las cookies para evitar volver a iniciar sesión:

In [2]:
# Configura tu navegador (en este caso, Chrome)
driver = webdriver.Chrome()

# Abre la página de Amazon
driver.get('https://www.amazon.es')

# Espera a que el usuario inicie sesión manualmente
input("Inicia sesión y presiona Enter para continuar...")

# Guarda las cookies en un archivo
cookies = driver.get_cookies()  # Obtiene todas las cookies
with open('amazon_cookies.pkl', 'wb') as f:
    pickle.dump(cookies, f)

print("Cookies guardadas correctamente.")

Inicia sesión y presiona Enter para continuar... 


Cookies guardadas correctamente.


### 2. Creamos funciones para ayudarnos en la extracción de las reseñas:

Obtiene las páginas HTML de un las reseñas de un producto, con un número de estrellas concreto.

In [3]:
def get_reviews_html(driver, product_id, star_filter):
    first_page_url = f"https://www.amazon.es/product-reviews/{product_id}/ref=acr_dp_hist_5?ie=UTF8&filterByStar={star_filter}&reviewerType=all_reviews#reviews-filter-bar"

    html_datas = []
    # Pedimos la página
    driver.get(first_page_url)
    time.sleep(5)  # Espera a que cargue la página

    for i in range (0, 11):
        # Guarda el HTML de la página actual
        html_datas.append(driver.page_source)

        try:
            # Esperar a que el botón "Siguiente" esté presente y sea clickeable
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "a-last"))
            )
            
            # Si el botón "Siguiente" está deshabilitado, hemos llegado al final
            if "a-disabled" in next_button.get_attribute("class"):
                print("No hay más páginas de reseñas.")
                break

            # Hacer clic en "Siguiente"
            next_button.click()
            time.sleep(5)  # Esperar a que cargue la nueva página
        except:
            print("No se encontró el botón 'Siguiente' o se llegó al final de las páginas.")
            break  # Salimos del bucle si no se encuentra el botón

    return html_datas

Dada una página HTML, devuelve las reseñas en formato BeautifulSoup.

In [4]:
def get_reviews_from_html(page_html: str) -> BeautifulSoup:
    soup = BeautifulSoup(page_html, "lxml")
    reviews = soup.find_all("div", {"class": "a-section celwidget"})
    return reviews

Dada una reseña en formato BeautifulSoup, devuelve la reseña en formato texto.

In [5]:
from langdetect import detect

def get_review_text(soup_object: BeautifulSoup) -> str:
    # Extraemos el texto de la reseña
    review_text = soup_object.find(
        "span", {"class": "a-size-base review-text review-text-content"}
    ).get_text()
    
    # Detectamos el idioma de la reseña
    try:
        lang = detect(review_text)
    except:
        lang = 'unknown'  # En caso de que no se pueda detectar el idioma
    
    # Filtramos solo las reseñas en español
    if lang == 'es':
        return review_text.strip()
    else:
        return None  # Si no está en español, devolvemos None (o podemos omitirla)



### 3. Obtenemos los ID-s de los productos que nos interesan sus reseñas:

In [23]:
ids = [
    "B0D87LPWWF"
]


### 4. Extraemos las reseñas:

In [8]:
!pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB 653.6 kB/s eta 0:00:06
   --- ------------------------------------ 0.3/3.8 MB 3.5 MB/s eta 0:00:01
   -------- ------------------------------- 0.8/3.8 MB 6.3 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 7.5 MB/s eta 0:00:01
   -------------------- ------------------- 1.9/3.8 MB 8.7 MB/s eta 0:00:01
   -------------------------- ------------- 2.6/3.8 MB 10.2 MB/s eta 0:00:01
   ----------------------------------- ---- 3.4/3.8 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------  3.8/3.8 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 10.6 MB/s eta 0:00:00


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time

# Configura el navegador
driver = webdriver.Chrome()

# Abre Amazon
driver.get('https://www.amazon.es')

# Cargar cookies si existen
try:
    with open('amazon_cookies.pkl', 'rb') as f:
        cookies = pickle.load(f)
        for cookie in cookies:
            driver.add_cookie(cookie)

    driver.refresh()
    print("Cookies cargadas correctamente.")
except FileNotFoundError:
    print("No se encontraron cookies guardadas. Inicia sesión manualmente.")

stars_mapping = {
    'one_star': 1,
    'two_star': 2,
    'three_star': 3,
    'four_star': 4,
    'five_star': 5,
    }

reviews = {1: [],
           2: [],
           3: [],
           4: [],
           5: []}

star_filters = ['one_star', 'two_star', 'three_star', 'four_star', 'five_star']
product_id = "B0D87LPWWF"

print(f"Extrayendo reseñas del producto.")
for star_filter in star_filters:
    print(f'Extrayendo reseñas del producto de {star_filter} estrellas...')
    # Obtenemos las páginas HTML de interes
    reviews_html = get_reviews_html(driver, product_id, star_filter)
    
    for html in reviews_html:
        soup_reviews = get_reviews_from_html(html)
        
        for soup_review in soup_reviews:
            review_text = get_review_text(soup_review)
            if review_text:  # Solo añade si no es None (es decir, si la reseña está en español)
                reviews[stars_mapping[star_filter]].append(review_text)
        
    
# Cierra el navegador
driver.quit()


Cookies cargadas correctamente.
Extrayendo reseñas del producto.
Extrayendo reseñas del producto de one_star estrellas...
No hay más páginas de reseñas.
Extrayendo reseñas del producto de two_star estrellas...
No se encontró el botón 'Siguiente' o se llegó al final de las páginas.
Extrayendo reseñas del producto de three_star estrellas...
No hay más páginas de reseñas.
Extrayendo reseñas del producto de four_star estrellas...
No hay más páginas de reseñas.
Extrayendo reseñas del producto de five_star estrellas...
No hay más páginas de reseñas.


In [7]:
print(len(reviews[2]))
# Contamos el número total de elementos en las listas dentro del diccionario
num_elementos_totales = sum(len(v) for v in reviews.values())
num_elementos_totales


4


118

### 5. Convertimos las reseñas en un DataFrame:

In [8]:
import pandas as pd

# Inicializamos listas vacías para reseñas y estrellas
review_list = []
stars_list = []

# Recorremos el diccionario y agregamos las reseñas y estrellas a las listas
for stars, review_texts in reviews.items():
    for review in review_texts:
        review_list.append(review)
        stars_list.append(stars)

# Creamos el DataFrame
reviews_df = pd.DataFrame({
    'review': review_list,
    'stars': stars_list
})

# Mostrar el DataFrame
print(reviews_df)


                                                review  stars
0    Es un bote para rociar aceite, mi opinión es q...      1
1    Lo siento , es fatal, no pulveriza nada bien. ...      1
2         No hace su función de spray. No lo aconsejo.      1
3    Funciona el primer día luego para tirarlo me d...      1
4             A los pocos usos se ha atascado el spray      1
..                                                 ...    ...
113  Muy cómodo y práctico. Rápida entrega y en el ...      5
114  Muy buena y fácil de usar ! Practica para pode...      5
115  Buen material, resistente y echa la cantidad p...      5
116  Bastante potencia y no desperdicia aceite, bue...      5
117  Apto para todo tipo de usos y producto de muy ...      5

[118 rows x 2 columns]


### 6. Guardamos el DataFrame en un archivo .csv:

In [10]:
# Guardar el DataFrame en un archivo CSV
reviews_df.to_csv('reviews_prod_20_29.csv', index=False)

### 7. Fusionamos los 3 datasets generados:

In [38]:
# Rutas de los datasets
path_1 = "data/reviews_prod_0_9.csv"
path_2 = "data/reviews_prod_10_19.csv"
path_3 = "data/reviews_prod_20_29.csv"

# Cargamos los datasets
data_1 = pd.read_csv(path_1)
data_2 = pd.read_csv(path_2)
data_3 = pd.read_csv(path_3)

# Fusionamos los tres DataFrames
data_combined = pd.concat([data_1, data_2, data_3], ignore_index=True)

print(data_combined.head())
# Imprimir la longitud (número de filas)
print("Número de filas:", len(data_combined))

                                              review  stars
0  900€ de IPhone para que a los 8 meses de compr...      1
1  He comprado este iPhone 14 y lo he puesto a 4 ...      1
2  Desde el momento 0 de tener este móvil me dio ...      1
3  Si, el producto está perfecto. Sería si habían...      1
4  Le he regalado a mi novia un iPhone 14 comprad...      1
Número de filas: 8441


In [39]:
# Guardamos el dataset combinado en un nuevo CSV
data_combined.to_csv("data/reviews.csv", index=False)